In [146]:
import cv2
import numpy as np
from sklearn import svm
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
import os



#pre_processed_img_dir='gear_images_converted/'
#pre_processed_label_dir= 'gear_labels_converted/'

pre_processed_img_dir = 'gear_images_converted'
pre_processed_label_dir = 'gear_labels_converted'

label_dict={'axes':0,'boots':1,'carabiners':2,'crampons':3,'gloves':4,'hardshell_jackets':5,'harnesses':6,'helmets':7
           ,'insulated_jackets':8,'pulleys':9,'rope':10,'tents':11}

In [147]:
def create_labels():
    for subdir,dirs, files in os.walk(pre_processed_img_dir):
    #print(dirs)
        for dir in dirs:
            if not os.path.exists(pre_processed_label_dir+'/'+dir): 
                os.makedirs(pre_processed_label_dir+'/'+dir )
            
            for file in os.listdir(pre_processed_img_dir+dir):
                f= open(pre_processed_label_dir+'/'+dir+'/'+file+'.txt',"w+")
                f.write(dir.replace('_',' '))


In [153]:
def train(x,y):

    #clf = svm.SVC(gamma='scale', decision_function_shape='ovo')
    #clf = SVC(gamma='auto')
    
    #clf = linear_model.SGDClassifier(max_iter=400)
    #clf.fit(x,y)
    clf=KNeighborsClassifier(n_neighbors=3)
    clf=clf.fit(x,y)

    return clf

In [154]:

def predict(clf,img):
    return(clf.predict(img))


In [155]:

added_img_array= []
added_label_array= []
for subdir, dirs, files in os.walk(pre_processed_img_dir):

    for dir in dirs:

        for file in os.listdir(pre_processed_img_dir + '/' + dir):

            img_array = cv2.imread(pre_processed_img_dir + '/' + dir + '/' + file)

            label_array=label_dict[dir]
            added_img_array.append(img_array)
            added_label_array.append(label_array)


added_label_array_array=np.delete(added_label_array,0)
added_label_array = np.asarray(added_label_array)
added_img_array = np.concatenate([arr[np.newaxis] for arr in added_img_array])
added_img_array = np.reshape(added_img_array, (added_img_array.shape[0], -1))

print(added_img_array.shape)
print(added_label_array.shape)

added_img_array=added_img_array.astype(float)
added_label_array=added_label_array.astype(float)



(2122, 49152)
(2122,)


In [156]:
added_label_array=np.transpose(added_label_array)
X_train, X_test, y_train, y_test = train_test_split(added_img_array, added_label_array, test_size=0.33, random_state=42)

clf = train(X_train,y_train)


In [157]:
truth=y_test.tolist()
results=[]
for x in X_test:
    result = predict(clf,x.reshape(1, -1))
    results.append(result)

print(accuracy_score(truth, results))


0.8145506419400856


In [161]:
testing_dir='challenge3-testimg_converted.jpeg'
testing_arr = cv2.imread(testing_dir)
testing_arr = np.reshape(testing_arr, (testing_arr.shape[0], -1))
print(testing_arr.shape)
res = predict(clf,testing_arr.reshape(1,-1))
print(res)

(128, 384)
[7.]
